본 파일은 알바몬 페이지를 크롤링 하기 위함임..
https://www.albamon.com/list/gi/mon_area_list.asp?page=2&ps=20&ob=6&lvtype=1&rArea=,B210&rWDate=1&Empmnt_Type=

이게 링크이며 이를 토대로 크롤링 진행!

In [79]:

from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import re
import requests

session = requests.Session()

title = [] #알바title = [] #알바 제목

result = pd.DataFrame(columns=['근무지역', '근무회사', '근무시간','급여','올린시간','알바설명'])  # 결과저장용 DataFrame
pagenum = 1 #첫 페이지
while(pagenum < 2):  #n개 페이지에 대하여 크롤링
    url = 'https://www.albamon.com/list/gi/mon_area_list.asp?page=1&ps=20&ob=6&lvtype=1&rArea=,B210&rWDate=1&Empmnt_Type=' #이건 알바천국
    url = url.replace('argu', str(pagenum))  # 페이지 1에대한 경우 이후 이는 변수로 처리할 것.

    pagenum = pagenum + 1 #다음페이지 준비
    page = session.get(url)
    c = page.content
    soup = BeautifulSoup(c, 'html.parser')
    title_ = soup.find_all('td')
    conte = soup.select_one('#subcontent > form > div.gListWrap > table > tbody')
    
    #지역관련 크롤링
    area = conte.select('td.area')
    for ww in area:
        print(ww.get_text().strip().replace('스크랩\n',""))
        print("-------------------------")
    #==================================================================

    #알바 제목
    cName = conte.select('td.subject > div.subWrap > p.cName')
    for ww in cName:
        print(ww.get_text())
        print("=====================")

    #==================================================================
    #내용

    cTit = conte.select('td.subject > div.subWrap > p.cTit')
    for ww in cTit:
        print(ww.get_text())
        print("===&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&====")


    #===================================================
    #급여
    pay = conte.select('td.pay')
    #지급방식은 돈의 범위로 한다.
    #시급 일급 월급으로 
    for ww in pay:
        print(ww.get_text().strip())
        print("-------------------------")

    #==================================================
    #근무시간
    time = conte.select('td:nth-child(4)')
    for ww in time:
        print(ww.get_text().strip())
        print("-####-----------------------")

    #================================================
    #올린시간
    recently = conte.select('td.recently > em')
    for ww in recently:
        print(ww.get_text().strip())
        print("-###23234-----------------------")



시흥시 정왕동
-------------------------
시흥시 정왕동
-------------------------
시흥시 정왕동
-------------------------
시흥시 신천동
-------------------------
시흥시 정왕동
-------------------------
시흥시 전체
-------------------------
시흥시 은행동
-------------------------
시흥시 정왕동
-------------------------
시흥시 정왕동
-------------------------
시흥시 배곧동
-------------------------
시흥시 월곶동
-------------------------
시흥시 대야동
-------------------------
시흥시 은행동
-------------------------
시흥시 전체
-------------------------
시흥시 배곧동
-------------------------
시흥시 정왕동
-------------------------
시흥시 정왕동
-------------------------
시흥시 정왕동
-------------------------
시흥시 전체
-------------------------
시흥시 전체
-------------------------
쿠팡풀필먼트서비스 유한회사
쿠팡풀필먼트서비스 유한회사
쿠팡물류 헬퍼-장,단기알바/굿윌아이엔씨㈜
쿠팡물류 헬퍼-장,단기알바/굿윌아이엔씨㈜
쿠팡물류 헬퍼-장,단기알바/굿윌아이엔씨㈜
부산은행
쿠팡CLS(시흥캠프)
㈜준시스템
쿠팡풀필먼트서비스 유한회사
쿠팡풀필먼트서비스 유한회사
쿠팡풀필먼트서비스 유한회사
쿠팡풀필먼트서비스 유한회사
쿠팡풀필먼트서비스 유한회사
[OPEN-일5시간] 광명센터-이벤트 고객상담
몽루아 BAR
차량운행해주세요
몽루아
쿠팡풀필먼트서비스(유)
오산쿠팡S팀 #이렇게돈벌기쉽지않은데#쉽지
오산쿠팡S팀 #이렇게돈벌기쉽지않은데#쉽지
[시흥쿠팡]#꿀알바#주휴#단기#장기#출